In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#importing dataframe
path = r'C:\Users\lenovo\09-2024 CareerFoundry'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'ords_prods_merge.pkl'))
ords_prods_merge.drop(['Unnamed: 0', 'Unnamed: 0.1','_merge'], axis=1, inplace=True)

In [3]:
df = ords_prods_merge[:1000000] #subsetting the dataframe

In [4]:
df.shape

(1000000, 14)

Q. Comparing the mean number of number of orders by department id across the subset and the entire dataframe

In [6]:
df.groupby('department_id')['order_number'].mean()

department_id
4     19.765396
7     18.575283
13    18.950838
14    20.460336
16    20.262396
17    12.646785
19    20.168036
20    18.808671
Name: order_number, dtype: float64

In [7]:
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     16.559358
2     18.413176
3     18.279600
4     18.915890
5     16.497751
6     17.609390
7     18.303975
8     16.383301
9     17.022963
10    21.227447
11    17.311768
12    16.953613
13    17.727470
14    17.898097
15    17.290123
16    18.760065
17    16.808683
18    20.336389
19    18.258620
20    17.578959
21    23.644477
Name: order_number, dtype: float64

We can see that more departments are represented in the entire dataframe than in the subset

Q. Creating a new column for flagging loyalty.
If number of orders is greater than 40, then 'Loyal customer'.
If number of orders is greater than 10 but less than or equal to 40, then 'Regular customer'
Otherwise, it's a 'New customer'

In [9]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [10]:
pd.options.display.max_rows = None

In [11]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

Q. Determine whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [13]:
#ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)
ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10094068.0,7.774068,4.206601,1.0,4.2,7.4,11.2,25.0
New customer,5150733.0,7.802315,4.267897,1.0,4.2,7.4,11.3,25.0
Regular customer,15079186.0,7.799374,4.255452,1.0,4.2,7.4,11.3,25.0


The mean prices charged for loyal customers is sligthly less than that charged for new and regular customers.

Q. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [15]:
#creating a column for each user based on average price
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'High spender'

Q. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [17]:
#creating a column for each user based on the median number of days since prior order
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [18]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [19]:
ords_prods_merge.head(50)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,match,max_order,loyalty_flag,mean_price,spending_flag,median_days_since_prior_order,frequency_flag
0,2398795,1,2,3,7,15,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
1,473747,1,3,3,12,21,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
2,2254736,1,4,4,7,29,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
3,431534,1,5,4,15,28,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
4,3367565,1,6,2,7,19,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
5,550135,1,7,1,9,20,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
6,3108588,1,8,1,14,14,196,2,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
7,2295261,1,9,1,16,0,196,4,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
8,2550362,1,10,4,8,30,196,1,1,Soda,77,7,9.0,both,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
9,2968173,15,15,1,9,7,196,2,0,Soda,77,7,9.0,both,22,Regular customer,3.994118,Low spender,10.0,Frequent customer


Exporting the dataframe to a .pkl file

In [20]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'orders_prods_frequency.pkl'))